In [ ]:
!pip install altair==5.3.0 vega_datasets pycountry --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.9 MB/s eta 0:00:00


In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.renderers.enable('html')

RendererRegistry.enable('html')

In [ ]:
import pandas as pd

df = pd.read_csv('Tabla.csv')
df.head()
df.columns

Index(['Año', 'Artista', 'Composición', 'Género', 'nacionalidad', 'escenario',
       'tiempo / escenario', 'headliner', 'Dia '],
      dtype='object')

In [ ]:
col_genero    = 'Género'
col_headliner = 'headliner'
col_pais      = 'nacionalidad'

In [ ]:
es_femenina  = df[col_genero].str.lower() == 'femenino'
es_headliner = df[col_headliner].str.lower().isin(['si', 'sí'])

In [ ]:

sub = df.loc[es_femenina & es_headliner, col_pais].str.strip()

print("Headliners femeninas encontradas:", len(sub))
sub.head(10)

Headliners femeninas encontradas: 10


,nacionalidad
235,Estados Unidos
243,España
284,Canada
288,Estados Unidos
305,Estados Unidos
353,Islandia
418,Estados Unidos
492,Estados Unidos
1049,Reino Unido
1099,Estados Unidos


In [ ]:
counts = sub.value_counts().reset_index()
counts.columns = ['pais_es', 'conteo']

print("Países distintos:", len(counts))
counts.head(10)

Países distintos: 5


,pais_es,conteo
0,Estados Unidos,6
1,España,1
2,Canada,1
3,Islandia,1
4,Reino Unido,1


In [ ]:
!pip install pycountry --quiet
import pycountry

fixes = {
    'estados unidos': 'USA',
    'reino unido'   : 'GBR',
    'españa'        : 'ESP',
    'canada'        : 'CAN',
    'méxico'        : 'MEX',
    'islandia'      : 'ISL',

}

def to_iso3(nombre):
    key = nombre.lower()
    if key in fixes:
        return fixes[key]
    try:
        return pycountry.countries.lookup(key).alpha_3
    except LookupError:
        return None

counts['iso3'] = counts['pais_es'].apply(to_iso3)

faltantes = counts['iso3'].isna().sum()
print("Países sin código ISO-3:", faltantes)
counts

Países sin código ISO-3: 0


,pais_es,conteo,iso3
0,Estados Unidos,6,USA
1,España,1,ESP
2,Canada,1,CAN
3,Islandia,1,ISL
4,Reino Unido,1,GBR


In [ ]:
import pycountry

counts['iso_n3'] = counts['iso3'].apply(
    lambda c: int(pycountry.countries.get(alpha_3=c).numeric)
)

In [ ]:
from vega_datasets import data as vds
import altair as alt

world = alt.topo_feature(vds.world_110m.url, 'countries')


base = (
    alt.Chart(world)
       .mark_geoshape(fill='#eeeeee', stroke='white')
       .properties(width=720, height=420)
)


data_layer = (
    alt.Chart(world)
       .mark_geoshape(stroke='white')
       .encode(
           color=alt.Color(
               'conteo:Q',
               scale=alt.Scale(
                   domain=[0, counts['conteo'].max()],
                   range=['#FFE2DE', '#e85343']
               ),
               legend=alt.Legend(title='Headliners')
           ),
           tooltip=[
               alt.Tooltip('pais_es:N', title='País'),
               alt.Tooltip('conteo:Q',  title='Headliners femeninas')
           ]
       )
       .transform_lookup(
           lookup='id',
           from_=alt.LookupData(
               counts, 'iso_n3', ['pais_es', 'conteo']
           )
       )
)


mapa = (
    (base + data_layer)
      .project(type='naturalEarth1')
      .properties(title='Headliners femeninas por país')
)

mapa

alt.LayerChart(...)

In [ ]:
mapa = (
    mapa
      .configure_view(fill='transparent', stroke=None)
      .configure(background='transparent')
)

mapa.save('headliners_mapa.html')